In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [85]:
c = 0.1

def cauchy_loss(x):
    global c
    f = np.log1p(np.square(x / c) / 2)

    return f

def smooth_l1(x):
    global c
    f = np.sqrt(np.square(x / c) + 1) - 1

    return f

def gemanmcclure(x):
    global c

    xx = np.square(x / c)
    f = 2 * xx / (xx + 4)
    return f

def welsch(x):
    global c

    xx = np.square(x / c)
    f = 1 - np.exp(-xx / 2)
    return f

x = np.linspace(-3.5 * c, 3.5 * c, 1000)


In [14]:
def plot_loss(function, title):
    plt.title(title)
    plt.grid(True)
    plt.plot(x, function(x))

In [26]:
functions = [cauchy_loss, smooth_l1, gemanmcclure, welsch]
titles = ["Cauchy Loss", "Another SmoothL1", "Geman-Mcclure Loss", "Welsch Loss"]

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True)
fig.suptitle("Robust Regresion Losses")

ax1.plot(x, cauchy_loss(x))
ax1.set_title("Cauchy Loss")

ax2.plot(x, smooth_l1(x))
ax2.set_title("Another SmoothL1 Loss")

ax3.plot(x, gemanmcclure(x))
ax3.set_title("Geman-McClure Loss")

ax4.plot(x, welsch(x))
ax4.set_title("Welsch Loss")

fig.tight_layout()

for ax in fig.get_axes():
    ax.label_outer()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, cauchy_loss(x), label=titles[0])
ax.plot(x, smooth_l1(x), label=titles[1])
ax.plot(x, gemanmcclure(x), label="Geman-McClure Loss")
ax.plot(x, welsch(x), label=titles[3])
ax.grid(True)
ax.legend()
plt.legend(bbox_to_anchor =(.93, 1.18), ncol = 2)
plt.rcParams['figure.dpi'] = 350
plt.rcParams['savefig.dpi'] = 350

In [2]:
from models import VAE
from dataset import CIFARDataset
from matplotlib import pyplot as plt
import numpy as np
import torch

In [ ]:
model = VAE()
model.load_state_dict(torch.load("autoencoder_weights/reconstruction_loss_best_autoencoder.pt", map_location=torch.device('cpu')))
dataset = CIFARDataset(train=False)

In [ ]:
samples = torch.Tensor(dataset[0][0]).unsqueeze(0)
model.eval()
samples = model(samples)
samples.shape

In [ ]:
#samples = model.sample(1, torch.device("cpu")).detach().numpy()
#samples = dataset[0][0]
samples = np.squeeze(samples.detach().numpy(), 0)
samples = samples.transpose(1, 2, 0)
plt.imshow(samples)

In [7]:
from blocks import CauchyLoss, WelschLoss, AnotherSmoothL1Loss, GemanMcClureLoss

In [ ]:
loss = GemanMcClureLoss()

with torch.autograd.detect_anomaly():
    x, y = torch.rand(1, 3, 32, 32, requires_grad=True), torch.rand(1, 3, 32, 32)
    out = loss(x, y)
    out.backward()

In [1]:
from utils import EmbeddingSearch, plot_3d
from sklearn.manifold import MDS

In [2]:
e = EmbeddingSearch()

Files already downloaded and verified


In [3]:
embeddings, labels = e.embeddings, e.labels

In [4]:
mds = MDS(n_components=3, verbose=1)

In [5]:
scaled = mds.fit_transform(embeddings)

In [7]:
from configs import base_config, autoencoder_config, classifier_config
import matplotlib
from matplotlib import cm, ticker
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [9]:
def vvisualize_embeddings(embeddings, labels, num_categories=classifier_config.num_classes):
    threshold = base_config.visualization_threshold
    if len(embeddings) > threshold:
        embeddings = embeddings[:threshold, :]
        labels = labels[:threshold]
    
    cmap = cm.get_cmap('tab20')
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    for label in range(num_categories):
        indexes = labels == label
        ax.scatter(embeddings[indexes, 0], embeddings[indexes, 1], embeddings[indexes, 2], c=np.array(cmap(label)).reshape(1, 4), label = label, alpha=0.5)
    
    plt.savefig(f'images/embeddings_{autoencoder_config.loss_function}.png')